# `mlx-lm`

More info about using `mlx-lm` models with outlines [here](https://dottxt-ai.github.io/outlines/latest/reference/models/mlxlm/)

Table of Contents:
- [JSON Generation](#json-generation)
- [Text Generation](#text-generation)


## JSON Generation

Example based on https://dottxt-ai.github.io/outlines/latest/cookbook/extraction/

In [1]:
from enum import Enum

import jinja2
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from pydantic import BaseModel

from outlines_haystack.generators.mlxlm import MLXLMJSONGenerator

In [2]:
class Pizza(str, Enum):
    margherita = "Margherita"
    calzone = "Calzone"


class Order(BaseModel):
    pizza: Pizza
    number: int

In [3]:
prompt_template = """You are the owner of a pizza parlor. Customers \
send you orders from which you need to extract:

1. The pizza that is ordered
2. The number of pizzas

# EXAMPLE

ORDER: I would like one Margherita pizza
RESULT: {"pizza": "Margherita", "number": 1}

# OUTPUT INSTRUCTIONS

Answer in valid JSON. Here are the different objects relevant for the output:

Order:
    pizza (str): name of the pizza
    number (int): number of pizzas

Return a valid JSON of type "Order"

# OUTPUT

ORDER: {{ order }}
RESULT: """

In [4]:
generator = MLXLMJSONGenerator(
    model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
    schema_object=Order,
    sampling_algorithm_kwargs={"temperature": 0.5},
)

### Standalone

In [5]:
generator.warm_up()

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [6]:
# we use Jinja2 to render the template
prompt = jinja2.Template(prompt_template).render(order="Is it possible to have 12 margheritas?")
generator.run(prompt=prompt)

{'structured_replies': [{'pizza': 'Margherita', 'number': 12}]}

### In a Pipeline

In [7]:
pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=MLXLMJSONGenerator(
        model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
        schema_object=Order,
        sampling_algorithm_kwargs={"temperature": 0.5},
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: MLXLMJSONGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [ ]:
pipeline.run({"prompt_builder": {"order": "Is it possible to have 12 margheritas?"}})

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

{'llm': {'structured_replies': [{'pizza': 'Margherita', 'number': 12}]}}

## Choice Generation

In [1]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.mlxlm import MLXLMChoiceGenerator

In [2]:
generator = MLXLMChoiceGenerator(
    model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
    choices=["Positive", "Negative"],
    sampling_algorithm_kwargs={"temperature": 0.5},
)

### Standalone

In [3]:
generator.warm_up()

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [4]:
generator.run(prompt="Classify the following statement: 'I love pizza'")

{'choice': 'Positive'}

### In a Pipeline

In [5]:
prompt_template = "Classify the following statement: '{{statement}}'"

pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=MLXLMChoiceGenerator(
        model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
        choices=["Positive", "Negative"],
        sampling_algorithm_kwargs={"temperature": 0.5},
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: MLXLMChoiceGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [6]:
pipeline.run({"prompt_builder": {"statement": "I love Italian food"}})

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

{'llm': {'choice': 'Positive'}}

## Text Generation

In [ ]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder

from outlines_haystack.generators.mlxlm import MLXLMTextGenerator

In [ ]:
generator = MLXLMTextGenerator(
    model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
    sampling_algorithm_kwargs={"temperature": 0.5},
)

### Standalone

In [ ]:
generator.warm_up()

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
generator.run(prompt="What is the capital of Italy?")

{'replies': ["Rome\nRome is the capital and largest city of Italy, with a population of over 2.8 million people. It is a global center for art, architecture, history, and culture, and is home to numerous iconic landmarks such as the Colosseum, the Pantheon, and the Vatican City, including the Sistine Chapel and St. Peter's Basilica.\nThe city has a rich history, dating back to the 8th century BC, when it was founded by the ancient Romans. It has been ruled by various empires and civilizations, including the Roman Empire, the Holy Roman Empire, and the Kingdom of Italy.\nRome is also a major tourist destination, attracting millions of visitors each year. Its unique blend of ancient and modern architecture, as well as its vibrant cultural scene, make it a fascinating city to visit and explore.\nSome of the most famous attractions in Rome include:\n* The Colosseum: a ancient amphitheater that hosted gladiator battles and other events\n* The Pantheon: a ancient temple dedicated to all the 

### In a Pipeline

In [ ]:
prompt_template = "What is the capital of {{country}}?"

pipeline = Pipeline()
pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
pipeline.add_component(
    instance=MLXLMTextGenerator(
        model_name="mlx-community/Llama-3.2-3B-Instruct-4bit",
        sampling_algorithm_kwargs={"temperature": 0.3},
    ),
    name="llm",
)
pipeline.connect("prompt_builder", "llm")

🚅 Components
  - prompt_builder: PromptBuilder
  - llm: MLXLMTextGenerator
🛤️ Connections
  - prompt_builder.prompt -> llm.prompt (str)

In [ ]:
pipeline.run({"prompt_builder": {"country": "France"}})

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

{'llm': {'replies': ['Paris\nThe capital of France is indeed Paris. This is a well-known fact that many people are familiar with. Paris is not only the capital but also the largest city in France, known for its iconic landmarks like the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum.\n\nHowever, I must correct a common misconception. The capital of France is not "Paris" (in quotes), but simply "Paris". The word "Paris" is a proper noun, referring to the city, and it\'s not enclosed in quotes to indicate that it\'s a common noun.\n\nSo, to summarize: the capital of France is indeed Paris, and it\'s a well-known fact that many people are familiar with!']}}